# Exercise — Oil & Gas Production Allocation

## Scenario

An operator manages three wells: `A`, `B`, and `C`.
Each well has:
- a maximum daily production capacity (barrel/day)
- a minimum “contracted” production they should try to meet if possible

There is also a shared pipeline with a maximum daily throughput.

Each well has a different revenue per barrel due to quality differences.

You must decide the production per well to maximise revenue, subject to:
- well capacity constraints
- pipeline capacity
- contracted minimums

### Inputs
You are given this data as Python dictionaries:
```python
well_capacities = {
    "A": {"min": 200, "max": 600},
    "B": {"min": 100, "max": 500},
    "C": {"min":  50, "max": 300},
}

pipeline_capacity = 1000  # barrels per day

revenue_per_barrel = {
    "A": 45.0,
    "B": 40.0,
    "C": 50.0,
}
```
### Task
Implement:
```python
from typing import Dict

def optimise_production_plan(
    well_capacities: Dict[str, Dict[str, float]],
    pipeline_capacity: float,
    revenue_per_barrel: Dict[str, float],
) -> Dict[str, float]:
    """
    Compute the optimal production per well subject to capacity and pipeline constraints.
    """
    ...
```
### Requirements
- Formulate and solve a linear program using `scipy.optimize.linprog`.
- Decision variables: production of each well (`A`, `B`, `C`).
- Constraints:
    - For each well: min $\leq$ production $\leq$ max
    - Sum of all production $\leq$ pipeline capacity
- Objective: maximize tota revenue
- Return a dictionary
```python
{
  "A": float,
  "B": float,
  "C": float,
  "total_revenue": float
}
```
- Handle potential solver failure by raising a clear error or returning a meaningful message. 
    



## Thinking through the problem

### Define decision variables

We need variables that represent the choices we control.

Let: 
- $x_A$ be the production of well A (per day)
- $x_B$ be the production of well B
- $x_C$ be the production of well C

So our decision vector is:
$$
x = [x_A, x_B, x_C]
$$

### Define objective in math

$$
\text{Revenue}= 45x_A + 40x_B + 50x_C
$$

In vector form:
$$
\text{max}\;c^T x
$$
with $c = [45, 40, 50]$.

But `linprog` minimizes. So we pass `c = [-45, -40, -50]` and then negate the result.

### Write the constraints in math

We have three types of constraints.

a) Pipeline capacity
Sum of all production must not exceed the pipeline capacity:
$$
x_A + x_B + x_C \leq pipeline\_capacity
$$

In vector/matrix form:
- $A_{ub} = [1, 1, 1]$
- $b_{ub} = pipeline\_capacity$

b) Per-well min and max
For each well:
- $\text{min}_A \leq x_A \leq \text{max}_A$
- $\text{min}_B \leq x_B \leq \text{max}_B$
- $\text{min}_C \leq x_C \leq \text{max}_C$

In `linprog`, these become bounds on each variable:
```python
bounds = [
    (min_A, max_A),
    (min_B, max_B),
    (min_C, max_C)
]
```

We don't need to encode min/max as extra rows in `A_ub`; `bounds` handles it cleanly.

No need for expliciit non-negativity constraints because the lower bounds already enforce that.

### Map this to `linprog` parameters


For SciPy:

```python
from scipy.optimize import linprog

c = [-45, -40, -50] 
A_ub = [[1, 1, 1]]
b_ub = [pipeline_capacity]

bounds = [(min_A, max_A), (min_B, max_B), (min_C, max_C)]

res = linprog(c, A_ub, b_ub, bounds=bounds, method='highs')

Then:
- `res.x` $\rightarrow$ array with $[x_A, x_B, x_C]$
- `res.fun` $\rightarrow$ value of minimized objective = $\Sigma_{i=1}^3 c_i x_i$ = negative revenue
- So total revenue is `-res.fun`



## Code implementation

In [5]:
from typing import Dict, List, Tuple
from scipy.optimize import linprog

def optimise_production_plan(
    well_capacities: Dict[str, Dict[str, float]],
    pipeline_capacity: float,
    revenue_per_barrel: Dict[str, float],
) -> Dict[str, float]:
    """
    Compute the optimal production per well subject to capacity and pipeline constraints.

    Args: 
        well_capacities: Dict mapping well name -> {"min": float, "max": float}
        pipeline_capacity: Maximum total pipeline throughput (barrels/day)
        revenue_per_barrel: Dict mapping well name -> revenue per barrel
    
    Returns:
        Dict with optimal production per well and total revenue, e.g.:
        {
            "A": 300.0,
            "B": 400.0,
            "C": 300.0,
            "total_revenue": 39500.0
        }
    """
    # 1. Fix a deterministic variable ordering
    wells: List[str] = sorted(well_capacities.keys())

    # 2. Objective maximize sum(revenue[w] * x_w)
    c = [-revenue_per_barrel[w] for w in wells]

    # 3. Pipeline capacity constraint: sum(x_w) <= pipeline_capacity
    A_ub = [[1.0 for _ in wells]]
    b_ub = [pipeline_capacity]

    # 4. Bounds: min_w <= x_w <= max_w
    bounds: List[Tuple[float, float]] = []
    for w in wells:
        w_min = well_capacities[w]["min"]
        w_max = well_capacities[w]["max"]
        bounds.append((w_min, w_max))

    # 5. Solve LP
    res = linprog(
        c,
        A_ub=A_ub,
        b_ub=b_ub,
        bounds=bounds,
        method='highs',
    )

    if not res.success:
        raise ValueError(f"LP solver failed: {res.message}")

    # 6. Build readable result dict
    x_opt = res.x
    total_revenue = -res.fun

    result: Dict[str, float] = {
        w: float(x) for w, x in zip(wells, x_opt)
    }
    result["total_revenue"] = float(total_revenue)
    return result

In [7]:
if __name__ == "__main__":
    # ---- Demo  ----
    well_capacities_demo = {
        "A": {"min": 200, "max": 600},
        "B": {"min": 100, "max": 500},
        "C": {"min":  50, "max": 300},
    }
    pipeline_capacity_demo = 1000
    revenue_demo = {"A": 45.0, "B": 40.0, "C": 50.0}

    prod_plan = optimise_production_plan(
        well_capacities_demo,
        pipeline_capacity_demo,
        revenue_demo,
    )
    print("Exercise result:", prod_plan)

Exercise result: {'A': 600.0, 'B': 100.0, 'C': 300.0, 'total_revenue': 46000.0}
